# Практическое задание к уроку 8 "Рекуррентные нейронные сети RNN LSTM GRU"

In [1]:

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import os
import re

DATA_PATH = '../data'

TRAIN_FILENAME = 'train.csv'
TEST_FILENAME = 'train.csv'
VALIDATION_FILENAME = 'train.csv'

df_train = pd.read_csv(os.path.join(DATA_PATH, TRAIN_FILENAME))
df_test = pd.read_csv(os.path.join(DATA_PATH, TEST_FILENAME))
df_val = pd.read_csv(os.path.join(DATA_PATH, VALIDATION_FILENAME))

In [2]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


### Выполняем препроцессинг

In [3]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [4]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [5]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [6]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [7]:
y_train = df_train['class'].values
y_val = df_val['class'].values

## Задание 1

Построить сверточные архитектуры

In [8]:
def build_convolutional_model(word_count, training_length=1000):
    model = Sequential()

    model = Sequential()
    model.add(Embedding(input_dim=word_count, output_dim=30, input_length=training_length))
    model.add(Conv1D(30, 3))
    model.add(Activation("relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(10))
    model.add(Activation("relu"))
    model.add(Dense(2))
    model.add(Activation('relu'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [9]:
conv_model = build_convolutional_model(word_count, training_length)

In [10]:
early_stopping=EarlyStopping(monitor='val_loss')
conv_history = conv_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 33s 79ms/step - loss: 1.7234 - accuracy: 0.4950 - val_loss: 0.6718 - val_accuracy: 0.5222


In [11]:
conv_score = conv_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', conv_score[0])
print('Test accuracy:', conv_score[1])

355/355 [==============================] - 0s 1ms/step - loss: 0.6481 - accuracy: 0.5465


Test score: 0.6480669379234314
Test accuracy: 0.5464630126953125


In [12]:
def build_rnn_model():
    model = Sequential()

    model.add(
        Embedding(
            input_dim=word_count,
            input_length=training_length,
            output_dim=30,
            trainable=True,
            mask_zero=True
        )
    )

    model.add(Masking(mask_value=0.0))

    model.add(SimpleRNN(64))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [13]:
rnn_model = build_rnn_model()

In [14]:
early_stopping=EarlyStopping(monitor='val_loss')
rnn_history = rnn_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
)

Epoch 1/10
319/319 [==============================] - 39s 106ms/step - loss: 0.6196 - accuracy: 0.6329 - val_loss: 0.4924 - val_accuracy: 0.7546


In [15]:
rnn_score = rnn_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', rnn_score[0])
print('Test accuracy:', rnn_score[1])

355/355 [==============================] - 3s 9ms/step - loss: 0.3439 - accuracy: 0.8625


Test score: 0.34392645955085754
Test accuracy: 0.8624818921089172


In [16]:
def build_lstm_model():
  model = Sequential()

  model.add(
      Embedding(input_dim=word_count,
                input_length=training_length,
                output_dim=30,
                trainable=True,
                mask_zero=True))
  model.add(Masking(mask_value=0.0))
  model.add(LSTM(64, recurrent_dropout=0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(
      optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [17]:
lstm_model = build_lstm_model()

In [18]:
early_stopping=EarlyStopping(monitor='val_loss')
lstm_history = lstm_model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=1,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
)

319/319 [==============================] - 51s 148ms/step - loss: 0.6090 - accuracy: 0.6508 - val_loss: 0.4972 - val_accuracy: 0.7536


In [19]:
lstm_score = lstm_model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', lstm_score[0])
print('Test accuracy:', lstm_score[1])

355/355 [==============================] - 5s 15ms/step - loss: 0.3747 - accuracy: 0.8519


Test score: 0.37469014525413513
Test accuracy: 0.8518794178962708


In [26]:
results = pd.DataFrame([conv_score, rnn_score, lstm_score], columns=['Test Score', 'Test Accuracy'])
results

,Test Score,Test Accuracy
0,0.648067,0.546463
1,0.343926,0.862482
2,0.374690,0.851879
